In [1]:
import os
import shutil
import yaml
from glob import glob
from ultralytics import YOLO
from roboflow import Roboflow

# --- 1. 라이브러리 임포트 ---
print("✅ 라이브러리 임포트 완료!")

✅ 라이브러리 임포트 완료!


In [2]:
# --- 2. Roboflow 데이터셋 다운로드 ---
print("🚀 Roboflow 데이터셋 다운로드를 시작합니다...")

# API 키 설정
rf = Roboflow(api_key="NjIXpou4o4gsuGClT8hI")

# 데이터셋 1: Acne-detection-21
project1 = rf.workspace("kritsakorn").project("acne-kbm0q")
version1 = project1.version(21)
dataset1 = version1.download("yolov11")
print(f"✅ 데이터셋 1 다운로드 완료: {dataset1.location}")

# 데이터셋 2: whitehead-1
project2 = rf.workspace("test-l2t0m").project("whitehead-kavgp")
version2 = project2.version(1)
dataset2 = version2.download("yolov11")
print(f"✅ 데이터셋 2 다운로드 완료: {dataset2.location}")

# 데이터셋 3: lesion-detection-ruc40
project3 = rf.workspace("skinlesions").project("lesion-detection-ruc40")
version3 = project3.version(10)
dataset3 = version3.download("yolov11")
print(f"✅ 데이터셋 3 다운로드 완료: {dataset3.location}")

print("-" * 50)

🚀 Roboflow 데이터셋 다운로드를 시작합니다...
loading Roboflow workspace...
loading Roboflow project...
✅ 데이터셋 1 다운로드 완료: c:\Users\Admin\work space\2nd\Acne-21
loading Roboflow workspace...
loading Roboflow project...
✅ 데이터셋 2 다운로드 완료: c:\Users\Admin\work space\2nd\whitehead-1
loading Roboflow workspace...
loading Roboflow project...
✅ 데이터셋 3 다운로드 완료: c:\Users\Admin\work space\2nd\lesion-detection-10
--------------------------------------------------


In [3]:
# --- 2단계: 경로 및 변수 설정 (수정) ---
BASE_DATA_PATH = 'C:/Users/Admin/work space/2nd/'
OLD_COMBINED_DATASET_NAME = 'acne_dataset_final'
NEW_DATASET_NAME = 'Acne-detection-21'
NEW_DATASET_2_NAME = 'whitehead-1'

# 👇 [추가] 새로 다운로드한 데이터셋 폴더 이름 (실제 다운로드된 이름으로 확인/수정)
NEW_DATASET_3_NAME = 'lesion-detection-10' # 예시 이름입니다. 실제 폴더명을 확인하세요.

# 👇 [수정] 모든 데이터를 합칠 최종 폴더 이름 (버전업)
FINAL_DATASET_NAME = 'skin_lesion_dataset_v4_combined'

In [4]:
# --- 3. 경로 및 변수 설정 ---
BASE_DATA_PATH = 'C:/Users/Admin/work space/2nd/'
OLD_COMBINED_DATASET_NAME = 'acne_dataset_final'
NEW_DATASET_1_NAME = 'Acne-21'
NEW_DATASET_2_NAME = 'whitehead-1'
NEW_DATASET_3_NAME = 'lesion-detection-10'
FINAL_DATASET_NAME = 'skin_lesion_dataset_v4_combined'

In [5]:
# --- 4. 클래스 필터링 및 데이터셋 재구성 ---
ORIGINAL_DATASET_NAME = 'skin_lesion_dataset_v4_combined'
original_path = os.path.join(BASE_DATA_PATH, ORIGINAL_DATASET_NAME)

# 선택된 5개 클래스 (높은 mAP 기준)
selected_classes = ['papule', 'comedone', 'nodule', 'cyst', 'whiteheads']

# 새 데이터셋 폴더
new_dataset_name = 'skin_lesion_dataset_selected_5'
new_path = os.path.join(BASE_DATA_PATH, new_dataset_name)
if os.path.exists(new_path):
    shutil.rmtree(new_path)
os.makedirs(new_path)

for split in ['train', 'valid', 'test']:
    os.makedirs(os.path.join(new_path, split, 'images'), exist_ok=True)
    os.makedirs(os.path.join(new_path, split, 'labels'), exist_ok=True)

# 원본 클래스 목록
original_classes = ['Acne', 'blackheads', 'comedone', 'cyst', 'eczema', 'fore', 'nodule', 'papule', 'psoriasis', 'pustule', 'rosacea', 'whiteheads']
original_name_to_id = {name: i for i, name in enumerate(original_classes)}
selected_name_to_id = {name: i for i, name in enumerate(selected_classes)}
remap_dict = {original_name_to_id[name]: selected_name_to_id[name] for name in selected_classes}

# 라벨 필터링 함수
def filter_and_remap_labels(src_label, dest_label):
    if os.path.exists(src_label):
        with open(src_label, 'r') as f:
            lines = f.readlines()
        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if parts:
                cls_id = int(parts[0])
                if cls_id in remap_dict:
                    new_cls_id = remap_dict[cls_id]
                    new_line = f"{new_cls_id} {' '.join(parts[1:])}\n"
                    new_lines.append(new_line)
        if new_lines:
            with open(dest_label, 'w') as f:
                f.writelines(new_lines)
            return True
    return False

# 이미지와 라벨 복사
for split in ['train', 'valid', 'test']:
    src_images = glob(os.path.join(original_path, split, 'images', '*.jpg')) + glob(os.path.join(original_path, split, 'images', '*.png'))
    for src_img in src_images:
        img_name = os.path.basename(src_img)
        src_label = os.path.join(original_path, split, 'labels', os.path.splitext(img_name)[0] + '.txt')
        dest_img = os.path.join(new_path, split, 'images', img_name)
        dest_label = os.path.join(new_path, split, 'labels', os.path.splitext(img_name)[0] + '.txt')
        if filter_and_remap_labels(src_label, dest_label):
            shutil.copy(src_img, dest_img)

print("✅ 선택 클래스 5개 데이터셋 생성 완료!")

# data.yaml 생성
data_yaml = {
    'path': new_path,
    'train': os.path.join(new_path, 'train', 'images'),
    'val': os.path.join(new_path, 'valid', 'images'),
    'test': os.path.join(new_path, 'test', 'images'),
    'nc': len(selected_classes),
    'names': selected_classes
}
new_yaml_path = os.path.join(BASE_DATA_PATH, 'data_selected_5.yaml')
with open(new_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)

print("✅ data.yaml 업데이트 완료!")

✅ 선택 클래스 5개 데이터셋 생성 완료!
✅ data.yaml 업데이트 완료!


In [6]:
# --- 5. 모델 재학습 ---
print("🚀 모델 재학습을 시작합니다...")

model = YOLO('yolov8s.pt')  # 스크래치 학습

results = model.train(
    data=new_yaml_path,
    epochs=200,
    imgsz=640,
    batch=16,
    project=BASE_DATA_PATH,
    name='skin_lesion_model_selected_5',
    exist_ok=True
)

print("\n🎉 재학습이 완료되었습니다!")
print("-" * 50)

🚀 모델 재학습을 시작합니다...
New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=C:/Users/Admin/work space/2nd/data_selected_5.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=200, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.

KeyboardInterrupt: 

In [ ]:
# --- 6. 모델 성능 검증 ---
print("🚀 최종 모델의 성능을 테스트 데이터셋으로 검증합니다...")

final_model_path = os.path.join(BASE_DATA_PATH, 'skin_lesion_model_selected_5', 'weights', 'best.pt')
model = YOLO(final_model_path)

metrics = model.val(data=new_yaml_path, split='test', imgsz=640)

print("\n--- ✅ 최종 성능 지표 ---")
print(f"mAP50-95: {metrics.box.map:.4f}")
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP75: {metrics.box.map75:.4f}")

🚀 최종 모델의 성능을 테스트 데이터셋으로 검증합니다...
Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 72 layers, 11,130,228 parameters, 0 gradients, 28.5 GFLOPs
val: Fast image access  (ping: 0.10.0 ms, read: 5.32.1 MB/s, size: 27.7 KB)
val: Scanning C:\Users\Admin\work space\2nd\skin_lesion_dataset_v4_combined\test\labels... 194 images, 35 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 229/229 265.5it/s 0.9s0.0s
val: New cache created: C:\Users\Admin\work space\2nd\skin_lesion_dataset_v4_combined\test\labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 15/15 3.6it/s 4.2s0.2s
                   all        229       2118       0.28      0.269      0.186     0.0628
                  Acne         24        768       0.35      0.165      0.143     0.0447
              comedone         73        471      0.355      0.325      0.277     0.0985
                  cyst    